In [1]:
import os
import time
import xml.etree.ElementTree as ET
from collections import OrderedDict

import numpy as np
import pandas as pd

In [2]:
# Data from:
# https://www.irs.gov/charities-non-profits/form-990-series-downloads

In [3]:
base = 'download990xml'

In [4]:
subs = []
for text in os.listdir(base):
    if text.startswith('download990xml'):
        subs.append(text)
        
subs.sort()
# display(subs)

In [5]:
print('Create an index file for each download folder:\n')
for i, sub in enumerate(subs):
    start = time.time()
    print(f'{i + 1:02} of {len(subs):02}, {sub:<21}', end = '')

    files = []
    for text in os.listdir(os.path.join(base, sub)):
        if text.endswith('public.xml'):
            files += [text]

    list0 =[]
    for file in files:
        with open(os.path.join(base, sub, file), 'r') as f: text = f.read()
        text = text.replace('xmlns="http://www.irs.gov/efile"', '')

        root = ET.fromstring(text)
        dic = OrderedDict()
        dic['Year'] = 0
        dic['Form'] = 0
        dic['State'] = 'XX'

        year1 = root.find('ReturnHeader/TaxYear')
        year2 = root.find('ReturnHeader/TaxYr')
        if year1 is not None: dic['Year'] = year1.text
        if year2 is not None: dic['Year'] = year2.text

        form1 = root.find('ReturnHeader/ReturnType')     
        form2 = root.find('ReturnHeader/ReturnTypeCd')
        if form1 is not None: dic['Form'] = form1.text
        if form2 is not None: dic['Form'] = form2.text

        state1 = root.find('ReturnHeader/Filer/USAddress/State')  
        state2 = root.find('ReturnHeader/Filer/USAddress/StateAbbreviationCd')
        if state1 is not None: dic['State'] = state1.text
        if state2 is not None: dic['State'] = state2.text
            
        path = os.path.join(sub, file)
        dic['Path'] = path
        
        list0.append(dic)

    df0 = pd.DataFrame(list0)
    index =  sub.replace('download990xml', 'index')
    df0.to_csv(os.path.join(base, index), index=False)
    
    stop = time.time()
    print(f', {(stop - start)/60:.2f} min')

Create an index file for each download folder:

01 of 45, download990xml_2015_1, 1.21 min
02 of 45, download990xml_2015_2, 1.09 min
03 of 45, download990xml_2016_1, 1.23 min
04 of 45, download990xml_2016_2, 1.25 min
05 of 45, download990xml_2016_3, 1.23 min
06 of 45, download990xml_2016_4, 1.24 min
07 of 45, download990xml_2016_5, 1.23 min
08 of 45, download990xml_2016_6, 0.74 min
09 of 45, download990xml_2017_1, 1.23 min
10 of 45, download990xml_2017_2, 1.24 min
11 of 45, download990xml_2017_3, 1.24 min
12 of 45, download990xml_2017_4, 1.24 min
13 of 45, download990xml_2017_5, 1.25 min
14 of 45, download990xml_2017_6, 1.24 min
15 of 45, download990xml_2017_7, 0.50 min
16 of 45, download990xml_2018_1, 1.26 min
17 of 45, download990xml_2018_2, 1.25 min
18 of 45, download990xml_2018_3, 1.25 min
19 of 45, download990xml_2018_4, 1.26 min
20 of 45, download990xml_2018_5, 1.25 min
21 of 45, download990xml_2018_6, 1.25 min
22 of 45, download990xml_2018_7, 0.92 min
23 of 45, download990xml_201

In [6]:
indexes = []
for text in os.listdir(base):
    if text.startswith('index_'):
        indexes.append(text)
        
indexes.sort()
# display(indexes)

In [7]:
list1 = []
for index in indexes:
    df = pd.read_csv(os.path.join(base, index))
    list1.append(df)
    
df1 = pd.concat(list1, ignore_index=True)
display(df1)
display(df1.info(null_counts=True))

,Year,Form,State,Path
0,2014,990PF,NJ,download990xml_2015_1/201513209349103101_publi...
1,2014,990,OH,download990xml_2015_1/201503209349307025_publi...
2,2014,990,NY,download990xml_2015_1/201513179349306981_publi...
3,2014,990PF,MI,download990xml_2015_1/201503009349100100_publi...
4,2014,990,VA,download990xml_2015_1/201503149349301075_publi...
...,...,...,...,...
2698283,2020,990,NC,download990xml_2022/202230359349301528_public.xml
2698284,2020,990,NY,download990xml_2022/202240749349300524_public.xml
2698285,2019,990,AZ,download990xml_2022/202200349349300930_public.xml
2698286,2020,990,PA,download990xml_2022/202230409349300203_public.xml


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2698288 entries, 0 to 2698287
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Year    2698288 non-null  int64 
 1   Form    2698288 non-null  object
 2   State   2698288 non-null  object
 3   Path    2698288 non-null  object
dtypes: int64(1), object(3)
memory usage: 82.3+ MB


None

In [8]:
df1.to_csv(os.path.join(base, 'index'), index=False)

In [9]:
display(df1['Year'].value_counts().sort_index(ascending=False))
display(df1['Form'].value_counts())
display(df1['State'].value_counts())

2020    289318
2019    478307
2018    476608
2017    439338
2016    429705
2015    369989
2014    211379
2013      3453
2012       187
0            4
Name: Year, dtype: int64

990      1500769
990EZ     774883
990PF     378123
990T       44509
0              4
Name: Form, dtype: int64

CA    316281
NY    237240
TX    160742
PA    136298
FL    132462
OH    110884
IL    101998
MA     89207
NJ     83499
MI     75609
VA     72684
NC     65745
GA     62762
MN     59039
WA     56661
WI     56453
CO     54005
MO     52371
MD     51781
IN     49976
TN     43728
CT     41316
OR     37592
IA     33251
AZ     32909
AL     31464
DC     31237
SC     30324
LA     28856
KS     26991
KY     26831
OK     25287
NE     23457
RI     20406
AR     19553
NV     19440
UT     17738
MS     16882
ME     16472
NM     15423
MT     15336
NH     15166
DE     13888
WV     13738
ID     12295
HI     12166
VT     11078
SD     10360
AK      9323
ND      8815
WY      8049
XX      1784
PR      1079
VI       291
GU        24
AP        24
AE         9
MP         6
AS         3
Name: State, dtype: int64

In [10]:
# The four files with year zero have an issue with the namespace.
# The issue can be corrected.  We ignore it for now.